In [ ]:
!pip install datasets
!pip install hazm
!pip install rouge
! pip install -U accelerate
! pip install -U transformers

In [1]:
import datasets
import pandas as pd
from IPython.display import display, HTML
from datasets import ClassLabel
from transformers import BertTokenizerFast
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments,AutoTokenizer
from transformers import EncoderDecoderModel
from transformers import BertTokenizer
from rouge import Rouge
import numpy as np
from hazm import *

2024-12-16 06:21:27.939096: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-16 06:21:27.939612: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-16 06:21:28.473865: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-16 06:21:29.963700: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-16 06:21:40.521366: W tensorflow/compiler/tf2

In [2]:
gpt_tokenizer=AutoTokenizer.from_pretrained("HooshvareLab/gpt2-fa",
    pad_token='<pad>',
    bos_token='<s>',
    eos_token='</s>',
    unk_token='<unk>')
tokenizer=AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")

In [3]:
gpt_tokenizer.pad_token = gpt_tokenizer.eos_token

In [4]:
train_data = datasets.load_dataset(
    "pn_summary", split="train", download_mode="force_redownload"
)

pn_summary.py:   0%|          | 0.00/5.55k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

pn_summary.zip:   0%|          | 0.00/89.6M [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.hf.co/datasets/pn_summary/49aa6a5fdb11244714f9bbe69517f2079ab934c9c565e272a977fbd8d2d404f7?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27pn_summary.zip%3B+filename%3D%22pn_summary.zip%22%3B&response-content-type=application%2Fzip&Expires=1734589323&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNDU4OTMyM319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9kYXRhc2V0cy9wbl9zdW1tYXJ5LzQ5YWE2YTVmZGIxMTI0NDcxNGY5YmJlNjk1MTdmMjA3OWFiOTM0YzljNTY1ZTI3MmE5NzdmYmQ4ZDJkNDA0Zjc%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIn1dfQ__&Signature=dLSFxEZ7FYVCG%7EcflcL17ofpXNIAEjTynSb7rGhhbB%7EWGhtJpmWHQNQlSXJojTczCsLDKB3Xz4oRh92nYnuJwf4IKzh8YBoq1gx3urj0Iw%7EXzEk2jBYQoYy1SfYOa3rH5EULYojoqH18OZucLliHSpkMmMQL2GTQzu7Ki70u2KCC2tTwPG4ryVh%7EVrift04qUyopCUscvEuyeP%7EpDID8GEEBkzfsb2ssJtIGqUO88qrlFr42wEOQ9FtHvQyVelo2gmojaZHw1G9V2xl6D7kTShiEQUsgv6Whz7SbFPc4QHuVz2ZSF4RuHJQfyt4IJOgPf7VD

pn_summary.zip:   0%|          | 0.00/89.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/82022 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5592 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5593 [00:00<?, ? examples/s]

In [6]:
normalizer = Normalizer()
def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    for i in range(len(batch['article'])):
      batch['article'][i]=normalizer.normalize(batch['article'][i])
      batch['summary'][i]=normalizer.normalize(batch['summary'][i])

    inputs = tokenizer(
        batch["article"],
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,
    )
    outputs = gpt_tokenizer(
        batch["summary"],
        padding="max_length",
        truncation=True,
        max_length=decoder_max_length,
    )
    outputs["input_ids"] = [
        [(label if label != gpt_tokenizer.pad_token_id else -100) for label in l]
        for l in outputs["input_ids"]
    ]
    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["labels"] = outputs.input_ids.copy()

    # because BERT automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`.
    # We have to make sure that the PAD token is ignored
    # batch["labels"] = [
    #     [1 if token == gpt_tokenizer.pad_token_id else token for token in labels]
    #     for labels in batch["labels"]
    # ]

    return batch

In [7]:
metric = Rouge()

In [25]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # Replace -100 in labels with pad_token_id for decoding
    labels_ids[labels_ids == -100] = gpt_tokenizer.pad_token_id

    # Decode predictions and labels
    decoded_preds = gpt_tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    decoded_labels = gpt_tokenizer.batch_decode(labels_ids, skip_special_tokens=True)
    # Compute ROUGE scores
    result = metric.get_scores(decoded_preds, decoded_labels,avg=True)
    print(result)
    with open("result.txt","a") as handler:
        handler.write(f'rouge-1 - recall: {result["rouge-1"]["r"]} - precision: {result["rouge-1"]["p"]} - fscore: {result["rouge-1"]["f"]}\n')
        handler.write(f'rouge-2 - recall: {result["rouge-2"]["r"]} - precision: {result["rouge-2"]["p"]} - fscore: {result["rouge-2"]["f"]}\n')
        handler.write(f'rouge-l - recall: {result["rouge-l"]["r"]} - precision: {result["rouge-l"]["p"]} - fscore: {result["rouge-l"]["f"]}\n\n')
    # Extract F-measure for each ROUGE score
    rouge_result = {
        "rouge1": result["rouge-1"]["f"],
        "rouge2": result["rouge-2"]["f"],
        "rougeL": result["rouge-l"]["f"],
    }

    return rouge_result

In [23]:
sample_size = 10000
encoder_max_length = 512
decoder_max_length = 128
#train_data = train_data.select(range(1000))
# batch_size = 16
batch_size = 4

In [10]:
train_data = train_data.map(
        process_data_to_model_inputs,
        batched=True,
        batch_size=batch_size,
        remove_columns=['id', 'title', 'article', 'summary', 'category', 'categories', 'network', 'link'],
    )

Map:   0%|          | 0/82022 [00:00<?, ? examples/s]

In [11]:
train_data.set_format(
        type="torch",
        columns=["input_ids", "attention_mask", "labels"],
    )

In [12]:
val_data = datasets.load_dataset(
        "pn_summary", split="validation"
    )

In [13]:
#val_data = val_data.select(range(200))

In [14]:
val_data = val_data.map(
        process_data_to_model_inputs,
        batched=True,
        batch_size=batch_size,
        remove_columns=['id', 'title', 'article', 'summary', 'category', 'categories', 'network', 'link'],
    )

Map:   0%|          | 0/5592 [00:00<?, ? examples/s]

In [15]:
val_data.set_format(
        type="torch",
        columns=["input_ids", "attention_mask", "labels"],
    )

In [16]:
bert2gpt = EncoderDecoderModel.from_encoder_decoder_pretrained(
        "HooshvareLab/bert-base-parsbert-uncased", "HooshvareLab/gpt2-fa"
    )
bert2gpt.save_pretrained("bert2bert")
bert2gpt = EncoderDecoderModel.from_pretrained("bert2bert")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/opt/conda/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variabl

In [17]:
bert2gpt.config.decoder_start_token_id = gpt_tokenizer.bos_token_id
bert2gpt.config.pad_token_id = gpt_tokenizer.pad_token_id
bert2gpt.config.vocab_size = bert2gpt.config.decoder.vocab_size
bert2gpt.generation_config.pad_token_id = gpt_tokenizer.eos_token_id
bert2gpt.config.max_length = 128
bert2gpt.config.min_length = 0
bert2gpt.config.no_repeat_ngram_size = 2
bert2gpt.config.early_stopping = True
bert2gpt.config.length_penalty = 2.0
bert2gpt.config.num_beams = 3

In [18]:
#gpt_tokenizer.eos_token_id
bert2gpt.generation_config.pad_token_id

2

In [26]:
training_args = Seq2SeqTrainingArguments(
        predict_with_generate=True,
        evaluation_strategy="steps",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        output_dir="bert2gpt_model",
        logging_steps=2,
        fp16=True,
        save_steps=10000,
        eval_steps=10000,
        num_train_epochs=5,
        report_to="none",
        warmup_steps=1000,
    )

/opt/conda/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Seq2SeqTrainer(
        model=bert2gpt,
        tokenizer=tokenizer,
        args=training_args,
        compute_metrics=compute_metrics,
        train_dataset=train_data,
        eval_dataset=val_data,
    )
trainer.train()
handler.close()

/tmp/ipykernel_301/4257497199.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/opt/conda/lib/python3.11/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:629: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
/opt/conda/lib/python3.11/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:649: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior

Step,Training Loss,Validation Loss


In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
device